# Python statsmodels for SARIMAX

This code is based on code from here: https://github.com/spierre91/builtiin/blob/main/time_series_forecasting.py

Some problems have been fixed. 

This current code base is not intended to be public or shared beyond ourselves.

In [ ]:
!pip install seaborn pandas==2.2.3 yfinance==0.2.48 pandas_datareader statsmodels scikit-learn

### Import dependencies and set up

In [ ]:
import pandas as pd 
import pandas_datareader as web 
import datetime

import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA


import pandas as pd
import yfinance as yf


import numpy as np
from sklearn.metrics import mean_squared_error




In [ ]:

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

sns.set_theme()

### Let's get some sample time-series data that we need

In [ ]:

btc = yf.download("AAPL", start='2018-01-01', end='2020-12-02')
btc['BTC-USD'] = btc['Close']
btc = btc[['BTC-USD']]
# btc = btc[['BTC-USD']].asfreq('M', method='bfill')



In [ ]:
btc.head()

### Let's split the dataset for back testing.

In [ ]:
pd.to_datetime("2020-11-01", format='%Y-%m-%d').tz_localize('UTC')

In [ ]:
btc.index[0]


In [ ]:
train = btc[btc.index < pd.to_datetime("2020-11-01", format='%Y-%m-%d').tz_localize('UTC')] 
test = btc[btc.index >= pd.to_datetime("2020-11-01", format='%Y-%m-%d').tz_localize('UTC')]


In [ ]:
train.head()

In [ ]:
test.head()

### Let's plot train and test data

In [ ]:

plt.ylabel('BTC Price')
plt.xlabel('Date')
plt.xticks(rotation=45)
plt.plot(train, color='black', label='Training')
plt.plot(test, color='red', label='Test')
plt.show()



## Train and test an ARMA model 
(autoregressive + moving average - no differencing)

In [ ]:

y = train['BTC-USD']

ARMAmodel = SARIMAX(y, order = (1, 0, 1)) # AR with 1 lookback, 0 differencing, and MA with 1 lookback
ARMAmodel = ARMAmodel.fit()

y_pred = ARMAmodel.get_forecast(len(test.index))
y_pred_df = y_pred.conf_int(alpha = 0.05) 
y_pred_df["Predictions"] = ARMAmodel.predict(start = y_pred_df.index[0], end = y_pred_df.index[-1])
y_pred_df.index = test.index
y_pred_arma = y_pred_df["Predictions"] 

arma_rmse = np.sqrt(mean_squared_error(test["BTC-USD"].values, y_pred_df["Predictions"]))
print(f"\n\n\nARMA RMSE: {arma_rmse}\n\n\n")



### Let's plot the output

In [ ]:

plt.ylabel('BTC Price')
plt.xlabel('Date')
plt.xticks(rotation=45)
plt.plot(train, color='black', label='Training')
plt.plot(test, color='red', label='Test')
plt.plot(y_pred_arma, color='green', label = 'ARMA Predictions')
plt.show()

## Train and test an ARIMA model 
(autoregressive + moving average - now with differencing)

In [ ]:
ARIMAmodel = ARIMA(y, order = (12, 1, 2)) # AR with 5 lookback, 1 differencing, and MA with 2 lookback
# ARIMAmodel = ARIMA(y, order = (7, 1, 4)) # AR with 7 lookback, 1 differencing, and MA with 4 lookback
# ARIMAmodel = ARIMA(y, order = (7, 2, 2)) # AR with 7 lookback, 2 differencing, and MA with 2 lookback
ARIMAmodel = ARIMAmodel.fit()

y_pred = ARIMAmodel.get_forecast(len(test.index))
y_pred_df = y_pred.conf_int(alpha = 0.05) 
y_pred_df["Predictions"] = ARIMAmodel.predict(start = y_pred_df.index[0], end = y_pred_df.index[-1])
y_pred_df.index = test.index
y_pred_arima = y_pred_df["Predictions"] 

arma_rmse = np.sqrt(mean_squared_error(test["BTC-USD"].values, y_pred_df["Predictions"]))
print(f"\n\n\nARIMA RMSE: {arma_rmse}\n\n\n")

### Let's plot the output of the ARIMA

In [ ]:

plt.ylabel('BTC Price')
plt.xlabel('Date')
plt.xticks(rotation=45)
plt.plot(train, color='black', label='Training')
plt.plot(test, color='red', label='Test')
plt.plot(y_pred_arma, color='green', label = 'ARMA Predictions')
plt.plot(y_pred_arima, color='yellow', label = 'ARIMA Predictions')
plt.show()

## Let's add seasonality (SARIMA)

In [ ]:

# SARIMAmodel = SARIMAX(y, order = (5, 4, 2), seasonal_order=(2,2,2,12))
SARIMAmodel = SARIMAX(y, order = (5, 1, 2), seasonal_order=(2,1,2,12)) # AR with 5 lookback, 4 differencing, and MA with 2 lookback, 12 month seasonality with AR of 2, differencing of 2, and MA of 2
#SARIMAmodel = SARIMAX(y, order = (5, 1, 2), seasonal_order=(2,1,2,365)) # AR with 5 lookback, 4 differencing, and MA with 2 lookback, 365 day seasonality with AR of 2, differencing of 2, and MA of 2
SARIMAmodel = SARIMAmodel.fit()

y_pred = SARIMAmodel.get_forecast(len(test.index))
y_pred_df = y_pred.conf_int(alpha = 0.05) 
y_pred_df["Predictions"] = SARIMAmodel.predict(start = y_pred_df.index[0], end = y_pred_df.index[-1])
y_pred_df.index = test.index
y_pred_sarimax = y_pred_df["Predictions"] 

arma_rmse = np.sqrt(mean_squared_error(test["BTC-USD"].values, y_pred_df["Predictions"]))
print(f"\n\n\nSARIMA RMSE: {arma_rmse}\n\n\n")

plt.show()

### Let's plot the output of SARIMA aganist the others

In [ ]:
plt.plot(train, color = "black", label = 'Training')
plt.plot(test, color = "red", label = 'Testing')
plt.ylabel('BTC Price')
plt.xlabel('Date')
plt.xticks(rotation=45)
plt.title("Train/Test split for BTC Data")



plt.plot(y_pred_arma, color='green', label = 'ARMA Predictions')
plt.plot(y_pred_arima, color='yellow', label = 'ARIMA Predictions')
plt.plot(y_pred_sarimax, color='blue', label = 'SARIMA Predictions')
plt.legend()

plt.show()